# Database Access with ClickHouse Connect

## Setup

Please install the following packages:
- clickhouse_connect
- python-dotenv

Need to open ssh tunnel to access remotely
- open terminal/command prompt
- run(with your username replaced): ssh -L 8123:localhost:8123 <user>@ppolak5.ams.stonybrook.edu

Your env file should look like this for remote access (just switch the comment if you're on campus):

#host= "ppolak5.ams.stonybrook.edu." 

host = "localhost"

server_user= "<server_username>"

server_password= "<server_password>"

db_user=  "<db_username>"

db_pass= "<db_password>"

In [ ]:
ssh -L 3306:localhost:3306 mhaggerty@ppolak5.ams.stonybrook.edu

# Custom SQL prompts to Clickhouse Database

## Trades

In [1]:
# import helper functions
from utils.clickhouse_query import *

# Note - there is restriction to 1,000,000 rows per day/per user - so it's wise to limit the query to a specific time range for testing purposes - aggregation can also be used to reduce the number of rows returned
# Here is a way to restrict the query to a specific time range
start_hour = 9
end_hour = 10

# Define the query - this query grabs trades data from AAPL on 2017-01-05 between 9am and 11am
query = f"""
SELECT * 
FROM TRADESDB.trades2017view 
WHERE (Symbol = 'AAPL') 
AND (Date = '2017-01-05') 
AND (toHour(Time) BETWEEN {start_hour} AND {end_hour})
AND Trade_Volume > 0
AND Trade_Price > 0
"""

# Execute the query and store the resulting dataframe
data = get_trades(query)

## Quotes

In [9]:
# import helper functions
from utils.clickhouse_query import *

# Note - there is restriction to 1,000,000 rows per day/per user - so it's wise to limit the query to a specific time range for testing purposes - aggregation can also be used to reduce the number of rows returned
# Here is a way to restrict the query to a specific time range
start_hour = 9
end_hour = 10

# Define the query - this query grabs quotes data from AAPL on 2017-01-05 between 9am and 11am
# get quotes
query = f'''
    SELECT * 
    FROM QUOTESDB.quotes2017
    WHERE Symbol = 'AAPL'
    AND Date = '2017-05-02'
    AND (toHour(Time) BETWEEN {start_hour} AND {end_hour})
'''
# Execute the query and store the resulting dataframe
quotes = get_quotes(query)

In [7]:
quotes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [1]:
# import helper functions
from utils.clickhouse_query import *

query = """
    SELECT *
    FROM QUOTESDB.quotes2017view
    LIMIT 10
"""
# Execute the query and store the resulting dataframe
data = get_quotes(query)

KeyError: 'Time'

In [ ]:
import pandas as pd

query = f'''
    SELECT * 
    FROM QUOTESDB.quotes2017view
    WHERE Symbol = 'AAPL'
    AND Date = '2017-05-02'
'''

# Execute the query and store the resulting dataframe
data = get_quotes(query)

# Export the dataframe to a CSV file
data.to_csv('/path/to/output/file.csv', index=False)

In [2]:
data.info( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47493 entries, 0 to 47492
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Time                                    47493 non-null  datetime64[ns]
 1   Exchange                                47493 non-null  object        
 2   Symbol                                  47493 non-null  object        
 3   Sale_Condition                          47493 non-null  object        
 4   Trade_Volume                            47493 non-null  int64         
 5   Trade_Price                             47493 non-null  float64       
 6   Trade_Stop_Stock_Indicator              47493 non-null  object        
 7   Trade_Correction_Indicator              47493 non-null  int64         
 8   Sequence_Number                         47493 non-null  int64         
 9   Trade_Id                                47493 non-

In [3]:
data

,Time,Exchange,Symbol,Sale_Condition,Trade_Volume,Trade_Price,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Source_of_Trade,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Date,YearMonth,DateTime
0,2017-01-05 09:00:20.099632006,P,AAPL,@ TI,1,116.03,nan,0,2744,77,N,,09:00:20.098955,NaN,0,2017-01-05,201701,2017-01-05 09:00:20.098955
1,2017-01-05 09:00:20.435260797,P,AAPL,@ TI,1,116.03,nan,0,2751,78,N,,09:00:20.434563,NaN,0,2017-01-05,201701,2017-01-05 09:00:20.434563
2,2017-01-05 09:01:08.877023201,K,AAPL,@FTI,42,116.07,nan,0,2795,48,N,,09:01:08.876758,NaN,1,2017-01-05,201701,2017-01-05 09:01:08.876758
3,2017-01-05 09:01:08.877479494,P,AAPL,@FTI,42,116.07,nan,0,2796,79,N,,09:01:08.876799,NaN,1,2017-01-05,201701,2017-01-05 09:01:08.876799
4,2017-01-05 09:01:08.884202687,P,AAPL,@FTI,66,116.07,nan,0,2797,80,N,,09:01:08.883539,NaN,1,2017-01-05,201701,2017-01-05 09:01:08.883539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47488,2017-01-05 10:59:59.949257478,K,AAPL,@F,200,116.33,nan,0,532527,6853,N,,10:59:59.944023,NaN,1,2017-01-05,201701,2017-01-05 10:59:59.944023
47489,2017-01-05 10:59:59.950371094,Z,AAPL,@F,100,116.33,nan,0,532529,4840,N,,10:59:59.944003,NaN,1,2017-01-05,201701,2017-01-05 10:59:59.944003
47490,2017-01-05 10:59:59.950377156,Z,AAPL,@F,100,116.33,nan,0,532530,4841,N,,10:59:59.944084,NaN,1,2017-01-05,201701,2017-01-05 10:59:59.944084
47491,2017-01-05 10:59:59.950397053,Z,AAPL,@F,300,116.33,nan,0,532531,4842,N,,10:59:59.944770,NaN,1,2017-01-05,201701,2017-01-05 10:59:59.944770


## Convert Timestamp to Participant Timestamp

In [67]:
trades = data

In [68]:
import pandas as pd

def convert_timestamp(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], format='%H%M%S%f').dt.time
    return df

trades = convert_timestamp(trades, 'Participant_Timestamp')

# Ensure "Date" column is in datetime format
trades['Date'] = pd.to_datetime(trades['Date'])

# Ensure "Participant_Timestamp" is of type str
trades['Participant_Timestamp'] = trades['Participant_Timestamp'].astype(str)

# Create new datetime column "DateTime"
trades['DateTime'] = pd.to_datetime(trades['Date'].dt.strftime('%Y-%m-%d') + ' ' + trades['Participant_Timestamp'])


In [74]:
trades

,Time,Exchange,Symbol,Sale_Condition,Trade_Volume,Trade_Price,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Source_of_Trade,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Date,YearMonth
DateTime,,,,,,,,,,,,,,,,,
2017-01-05 09:00:20.098955,2017-01-05 09:00:20.099632006,P,AAPL,@ TI,1,116.03,nan,0,2744,77,N,,09:00:20.098955,NaN,0,2017-01-05,201701
2017-01-05 09:00:20.434563,2017-01-05 09:00:20.435260797,P,AAPL,@ TI,1,116.03,nan,0,2751,78,N,,09:00:20.434563,NaN,0,2017-01-05,201701
2017-01-05 09:01:08.876758,2017-01-05 09:01:08.877023201,K,AAPL,@FTI,42,116.07,nan,0,2795,48,N,,09:01:08.876758,NaN,1,2017-01-05,201701
2017-01-05 09:01:08.876799,2017-01-05 09:01:08.877479494,P,AAPL,@FTI,42,116.07,nan,0,2796,79,N,,09:01:08.876799,NaN,1,2017-01-05,201701
2017-01-05 09:01:08.883539,2017-01-05 09:01:08.884202687,P,AAPL,@FTI,66,116.07,nan,0,2797,80,N,,09:01:08.883539,NaN,1,2017-01-05,201701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-05 10:59:59.944023,2017-01-05 10:59:59.949257478,K,AAPL,@F,200,116.33,nan,0,532527,6853,N,,10:59:59.944023,NaN,1,2017-01-05,201701
2017-01-05 10:59:59.944003,2017-01-05 10:59:59.950371094,Z,AAPL,@F,100,116.33,nan,0,532529,4840,N,,10:59:59.944003,NaN,1,2017-01-05,201701
2017-01-05 10:59:59.944084,2017-01-05 10:59:59.950377156,Z,AAPL,@F,100,116.33,nan,0,532530,4841,N,,10:59:59.944084,NaN,1,2017-01-05,201701


In [75]:
import pandas as pd

# Assuming 'df' is your DataFrame
trades['Time - PTS'] = pd.to_datetime(trades['DateTime'])  # Ensuring DateTime column is of datetime type
trades.set_index('Time - PTS', inplace=True)  # Setting DateTime column as index


KeyError: 'DateTime'

In [70]:
trades

,Time,Exchange,Symbol,Sale_Condition,Trade_Volume,Trade_Price,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Source_of_Trade,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator,Date,YearMonth
DateTime,,,,,,,,,,,,,,,,,
2017-01-05 09:00:20.098955,2017-01-05 09:00:20.099632006,P,AAPL,@ TI,1,116.03,nan,0,2744,77,N,,09:00:20.098955,NaN,0,2017-01-05,201701
2017-01-05 09:00:20.434563,2017-01-05 09:00:20.435260797,P,AAPL,@ TI,1,116.03,nan,0,2751,78,N,,09:00:20.434563,NaN,0,2017-01-05,201701
2017-01-05 09:01:08.876758,2017-01-05 09:01:08.877023201,K,AAPL,@FTI,42,116.07,nan,0,2795,48,N,,09:01:08.876758,NaN,1,2017-01-05,201701
2017-01-05 09:01:08.876799,2017-01-05 09:01:08.877479494,P,AAPL,@FTI,42,116.07,nan,0,2796,79,N,,09:01:08.876799,NaN,1,2017-01-05,201701
2017-01-05 09:01:08.883539,2017-01-05 09:01:08.884202687,P,AAPL,@FTI,66,116.07,nan,0,2797,80,N,,09:01:08.883539,NaN,1,2017-01-05,201701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-05 10:59:59.944023,2017-01-05 10:59:59.949257478,K,AAPL,@F,200,116.33,nan,0,532527,6853,N,,10:59:59.944023,NaN,1,2017-01-05,201701
2017-01-05 10:59:59.944003,2017-01-05 10:59:59.950371094,Z,AAPL,@F,100,116.33,nan,0,532529,4840,N,,10:59:59.944003,NaN,1,2017-01-05,201701
2017-01-05 10:59:59.944084,2017-01-05 10:59:59.950377156,Z,AAPL,@F,100,116.33,nan,0,532530,4841,N,,10:59:59.944084,NaN,1,2017-01-05,201701


In [1]:
# import helper functions
from utils.clickhouse_query import *
from utils.data import *
# Select apple trades from January of 2017 to April of 2017
query = f"""
SELECT * 
FROM TRADESDB.trades2017view 
WHERE (Symbol = 'AAPL') 
AND (Date = '2017-01-05') 
AND (toHour(Time) BETWEEN {start_hour} AND {end_hour})
AND Trade_Volume > 0
AND Trade_Price > 0
"""
data = load_and_preprocess_data(query)

ValueError: unconverted data remains: 000000